

```
Go to the directory of your dataset which contains the images and csv file for images
```



In [ ]:
%cd /content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset

/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset


```
Clone the paddleocr folder in your dataset directory[only first time required to clone]
```


In [ ]:
# !git clone https://github.com/PaddlePaddle/PaddleOCR.git

```
Go to the PaddleOCR Folder
```

In [ ]:
%cd PaddleOCR

```
Install all the library present inside the requirements.txt file into the PaddleOCR Folder[Before execution this verify that paddlepaddle-gpu library also present into the requirements.txt otherwise add it]
```

In [ ]:
!pip install -r requirements.txt

  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.8.0.76
    Uninstalling opencv-contrib-python-4.8.0.76:
      Successfully uninstalled opencv-contrib-python-4.8.0.76


```
PaddleOCR required the label file in particular format and this format is create by the PaddleOCR itself. Below script will generate the particular format automatically.
```

```
input path : Path of the csv file which contains the file_name with extension and numeric value available in this particular image.
```
```
output path : Where the generated txt file is stored.
Columns name exclude.
```

In [ ]:
# convert the official gt to rec_gt_label.txt
!python ppocr/utils/gen_label.py --mode="rec" --input_path="/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset/with_extension.csv" --output_label="output_label1.txt"

Generate rec label


```
Download the pretrained model from the official website of paddleocr
```

```
Decompress the downloaded tar file into the original form
```

In [ ]:
%cd PaddleOCR/
# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
# Decompress model parameters
%cd pretrain_models
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar

```
Install the library of paddle for gpu use cases
!pip install paddlepaddle-gpu
For cpu execution other library required.
```

In [ ]:
!pip install paddlepaddle-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.1 MB/s eta 0:00:00


```
Perform the fine-tuning of the model.
Update the yml file as per your requirement and paste the path of updated file.
```

In [ ]:
!python3 -m paddle.distributed.launch  tools/train.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=/content/drive/MyDrive/paddle_ocr/handwritten_digit_dataset/PaddleOCR/pretrain_models/en_PP-OCRv3_rec_train/best_accuracy.pdparams

LAUNCH INFO 2024-05-03 09:32:31,181 -----------  Configuration  ----------------------
LAUNCH INFO 2024-05-03 09:32:31,181 auto_parallel_config: None
LAUNCH INFO 2024-05-03 09:32:31,181 auto_tuner_json: None
LAUNCH INFO 2024-05-03 09:32:31,181 devices: None
LAUNCH INFO 2024-05-03 09:32:31,181 elastic_level: -1
LAUNCH INFO 2024-05-03 09:32:31,181 elastic_timeout: 30
LAUNCH INFO 2024-05-03 09:32:31,181 enable_gpu_log: True
LAUNCH INFO 2024-05-03 09:32:31,181 gloo_port: 6767
LAUNCH INFO 2024-05-03 09:32:31,181 host: None
LAUNCH INFO 2024-05-03 09:32:31,181 ips: None
LAUNCH INFO 2024-05-03 09:32:31,181 job_id: default
LAUNCH INFO 2024-05-03 09:32:31,181 legacy: False
LAUNCH INFO 2024-05-03 09:32:31,181 log_dir: log
LAUNCH INFO 2024-05-03 09:32:31,181 log_level: INFO
LAUNCH INFO 2024-05-03 09:32:31,181 log_overwrite: False
LAUNCH INFO 2024-05-03 09:32:31,182 master: None
LAUNCH INFO 2024-05-03 09:32:31,182 max_restart: 3
LAUNCH INFO 2024-05-03 09:32:31,182 nnodes: 1
LAUNCH INFO 2024-05-03 0

In [ ]:
%cd PaddleOCR

```
Below Script will export the model into the file so that we can use for the inference
```

In [ ]:
#Conversion of Recognizer Trained weights to Inference
!python3 tools/export_model.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=output/v3_en_mobile/latest  Global.save_inference_dir=./inference/en_PP-OCRv3_rec/

```
Checking whether the paddle library is comptible with GPU or CPU
```

In [ ]:
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


```
Code for see the output for particular image
```

In [ ]:
from paddleocr import PaddleOCR,draw_ocr


```
rec_model_dir = file path where the model export.
Based on CPU or GPU specify the use_gpu = True/False.
```

In [ ]:
custom_ocr = PaddleOCR(use_angle_cls=True,
                rec_model_dir='./inference/en_PP-OCRv3_rec',
                det_model_dir='./output/det_db_inference',
                rec_char_dict_path='ppocr/utils/en_dict.txt',
                use_gpu=True,
                show_log=False,
                cls=False,
                det=True)

download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:14<00:00, 152kiB/s]


```
Specify the path of the image
```

In [ ]:
import cv2
img_path = "/content/is01001.bmp"
img = cv2.imread(img_path)
result = custom_ocr.ocr(img)
result

[[[[[34.0, 49.0], [94.0, 49.0], [94.0, 86.0], [34.0, 86.0]],
   ('1901', 0.9660794734954834)],
  [[[556.0, 42.0], [1039.0, 42.0], [1039.0, 89.0], [556.0, 89.0]],
   ('19191910101019209106119', 0.7923064827919006)],
  [[[32.0, 92.0], [196.0, 106.0], [193.0, 148.0], [29.0, 134.0]],
   ('19112021217', 0.7559323906898499)],
  [[[555.0, 104.0], [610.0, 111.0], [606.0, 148.0], [550.0, 141.0]],
   ('1923', 0.7396922707557678)],
  [[[711.0, 95.0], [908.0, 101.0], [907.0, 150.0], [710.0, 144.0]],
   ('151190199112', 0.8296001553535461)],
  [[[991.0, 100.0], [1078.0, 100.0], [1078.0, 142.0], [991.0, 142.0]],
   ('190179', 0.7904401421546936)],
  [[[1390.0, 100.0], [1542.0, 100.0], [1542.0, 142.0], [1390.0, 142.0]],
   ('1011501', 0.6909913420677185)],
  [[[198.0, 163.0], [411.0, 163.0], [411.0, 220.0], [198.0, 220.0]],
   ('1901910195', 0.8578236699104309)],
  [[[48.0, 173.0], [182.0, 173.0], [182.0, 222.0], [48.0, 222.0]],
   ('199503', 0.7587192058563232)],
  [[[1020.0, 164.0], [1194.0, 158.0]